# SVD compression

In order to understand the truncating function of a singular value decomposition (SVD) a bit better, we will have a look at the its application on image data. We treat the image as a matrix and apply the SVD to this data.

## Introduction to SVD

The singular value decomposition (SVD) is a matrix operation that splits a matrix into two unitary matrices $U$ and $V^\dagger$ and a diagonal matrix $S$ which contains the singular values.

![SVD](img_notebook/svd.png)

The decomposition splits an arbitrary $m$ x $n$ matrix M into three parts:
- $m$ x $m$ unitary $U$
- $m$ x $n$ diagonal matrix $S$
- $n$ x $n$ diagonal matrix $V^\dagger$

The diagonal matrix $S$ is padded with zeros such that it fits the desired shape.

A straight forward way to truncate a matrix is to truncate the matrices in the following way:

![truncated SVD](img_notebook/svd_trunc.png)

Note that the shape of the matrices does not change, since the dimensions that are relevant for the overall shape stay untouched.

Import of some useful modules

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

Load an image using PIL and convert it to an black and white image.

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
import os
from os.path import join
image_list=[(f,join('images',f)) for f in os.listdir('images')]
filename="joshua.jpg"
def set_filename(image):
    global filename
    filename=image
interact(set_filename,image=image_list);

In [ ]:
img=Image.open(filename).convert('L')
img_np=np.asarray(img)
#Plot the original image in grayscale
fig, axes=plt.subplots()
axes.imshow(img_np,cmap=plt.cm.gray);

Apply the SVD to the image, truncate it and display the result. Make sure that the largest singular values are taken into account when you truncate.

In [ ]:
trunc=20
#Apply the SVD to the array
u,s,vh=np.linalg.svd(img_np)
reconstruction=u[:,:trunc]@np.diag(s[:trunc])@vh[:trunc,:]
#Plot the reconstruction with reduced number of singular values
fig,axes=plt.subplots()
axes.imshow(reconstruction,cmap=plt.cm.gray);

We use the interactive possibilities of Jupyter Notebooks to explore the interplay between image quality and the number of singular values that are taken into account.

In [ ]:
fig,axes=plt.subplots()
contents=axes.imshow(img, cmap=plt.cm.gray)

def update_image_truncated(dim):                                                                                                                                                                                                                                                           
    reconstruction=u[:,:dim]@np.diag(s[:dim])@vh[:dim,:]
    contents.set_data(reconstruction)
interact(update_image_truncated,dim=IntSlider(min=2,max=512,continuous_update=False));

## Extra information about SVD

Since we want our tensors to be as small as possible but still as exact as possible, we are interested in a low-rank approximation of a possibly larger matrix.
It was shown by Eckart and Young in 1936 [1] that approximating the matrix $M$ with 
\begin{align}
    \tilde{M}^{(k)}=U S^{(k)} V^\dagger,
\end{align}
is optimal with respect to the Frobenius norm, i.e. $\lVert M-\tilde{M}^{(k)} \rVert\leq \lVert M-Y\rVert$ for all $Y$ of rank $k$. The notation $(k)$ refers to taking the largest $k$ singular values into account. 

[1] C. Eckart and G. Young, “The approximation of one matrix by another of lower rank,” Psychometrika 1, (1936)